In [ ]:
%matplotlib widget
"""Program to test the LRP algorithm implemented in Tensorflow using pre-trained VGG models.
"""
from functions import *
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or '3' for FATAL logs only

In [ ]:
nr = 21
Capacity_forecasting = 1 # 1: Capacity forecasting, 0: traffic forecasting

if Capacity_forecasting == 1:
    damage = 'capacity_forecasting'
else:
    damage = 'mae'

cell = 5060
cells = get_rows(cell, nr)

pathdatafolder = "../Trained_models/"+str(damage)+"/Data/"
pathmodelfolder = "../Trained_models/"+str(damage)+"/Models/"

In [ ]:
for cell_id in cells:

    print('Cell: ', cell_id)

    # get rows of the city
    digits = get_rows(cell_id, 5)

    # Output folder where selected files will be copied
    subdata = "../Trained_models/"+str(damage)+"/data_"+str(damage)+"/"
    submodels = "../Trained_models/"+str(damage)+"/models_"+str(damage)+"/"

    # Check if the submodels and subdata directories exist
    if os.path.exists(submodels):
        shutil.rmtree(submodels)  # delete all the files inside the directory
    if os.path.exists(subdata):
        shutil.rmtree(subdata)  # delete all the files inside the directory

    # Create the submodels and subdata folders
    os.makedirs(submodels)
    os.makedirs(subdata)

    # Iterate over the digits and copy the corresponding files to the subfolders
    for digit in digits:
        # Find the model and data files with the current digit
        model_files = [f for f in os.listdir(pathmodelfolder) if f.endswith('_{}.h5'.format(digit))]
        data_files = [f for f in os.listdir(pathdatafolder) if f.endswith('_{}.npy'.format(digit))]

        # Copy the model and data files to the subfolders
        for model_file in model_files:
            shutil.copy(os.path.join(pathmodelfolder, model_file), submodels)

        for data_file in data_files:
            shutil.copy(os.path.join(pathdatafolder, data_file), subdata)

    nr = 5

    final_dims = [1780, 3, nr + 4, nr + 4]

    disable_eager_execution()
    lookback = final_dims[1]
    model_dims = [3, 5, 5]
    final_image = np.zeros(final_dims)

    datasets_l = sorted(os.listdir(subdata))
    models_l = sorted(os.listdir(submodels))
    conf = yaml.safe_load(open("config.yml"))
    col = 0
    row = 0
    for ind, (dataset, model) in enumerate(zip(datasets_l, models_l)):
        path = subdata + dataset
        pathmodel = submodels + model
        data, model = prep_data(path, pathmodel)
        data = extract_windows(data, lookback, final_dims[0], lookback)
        image = data[0:final_dims[0]]
        lrpim = layer_wise_relevance_propagation(model, conf, image, ind)
        K.clear_session()
        lrpim = np.squeeze(lrpim, axis=4)
        final_image[:, :, row:row + model_dims[1],
                    col:col + model_dims[2]] += lrpim

        col += 1
        if (col + model_dims[1]) > final_dims[2]:
            col = 0
            row += 1
    final_image = final_image[:, :, int(model_dims[1] / 2):final_dims[2] - int(
        model_dims[1] / 2), int(model_dims[2] / 2):final_dims[3] - int(model_dims[2] / 2)]

    x = moving_average(final_image, 1)
    xx = []
    for t in range(0,x.shape[0]):
        x1 = 0.2*x[t,0] + 0.7*x[t,1] + 0.1*x[t,2]
        xx.append(x1)
    xx = np.asarray(xx)
    print(xx.shape)

    # get the most important cell at each time instance and store in an array
    t3 = []
    N = 1
    id2 = []
    for t in range(0,x.shape[0]):
        t3 = (xx[t,:,:])
        t3[2,2]=0
        t3 = t3.flatten()
        t3 = t3.argsort()[-N:][::-1]
        id = t3
        id2 = np.append(id,id2)

    id2 = id2.reshape(x.shape[0],1)
    np.save('cell_ids/'+str(cell_id)+'_'+str(damage)+'_most_relevant.npy', id2)

    # get the least important cell at each time instance and store in an array
    t3 = []
    N = 25
    id2 = []
    for t in range(0,x.shape[0]):
        t3 = (xx[t,:,:])
        t3[2,2] = 10
        t3 = t3.flatten()
        t3 = t3.argsort()[-N:][::-1]
        t3 = t3[24]
        id = t3
        id2 =np.append(id,id2)

    id2 = id2.reshape(x.shape[0],1)
    np.save('cell_ids/'+str(cell_id)+'_'+str(damage)+'_least_relevant.npy', id2)
